# This is a notebook for some testing

In [6]:
import os
import glob
import importlib.util

# Get the current working directory
project_dir = os.getcwd()

# List all Python files in the project directory
python_files = glob.glob(os.path.join(project_dir, '**', '*.py'), recursive=True)

print(python_files)

# import the python_files as modules
for file in python_files:
    module_name = os.path.splitext(os.path.basename(file))[0]


['c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\capfacloc_model.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\coder_agent.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\contrast_extract_mps_info.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\main.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\run.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\scenario_planner.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\sensitivity_analysis.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\simple_model.py', 'c:\\Users\\timpi\\Documents\\Ugent\\Masterproef\\multi_agent_supply_chain_optimization\\traditional_sensitivity_analysis.py', 'c:\\U

In [15]:
from contrast_extract_mps_info import analyze_mps, analyze_mps_high_level
import pandas as pd

In [24]:
# Example usage:
file_path = os.path.join(project_dir, "CapacitatedFacilityLocation.mps")
mps_data = analyze_mps(file_path)

df = pd.DataFrame.from_dict(mps_data, orient='index')
df = df.transpose()
df.head(20)
# print a subset of df
#df.sample(10)

,objective_function,constraints,rhs_values
name,TotalCost,NaN,NaN
coefficients,"{'Open_0': 150.0, 'Open_1': 180.0, 'Open_2': 1...",NaN,NaN
Demand_0,NaN,"{'type': 'G', 'coefficients': {'Serve_0_0': 1....",1.0
Demand_1,NaN,"{'type': 'G', 'coefficients': {'Serve_1_0': 1....",1.0
Demand_2,NaN,"{'type': 'G', 'coefficients': {'Serve_2_0': 1....",1.0
Demand_3,NaN,"{'type': 'G', 'coefficients': {'Serve_3_0': 1....",1.0
Demand_4,NaN,"{'type': 'G', 'coefficients': {'Serve_4_0': 1....",1.0
Capacity_0,NaN,"{'type': 'L', 'coefficients': {'Open_0': -80.0...",0.0
Capacity_1,NaN,"{'type': 'L', 'coefficients': {'Open_1': -90.0...",0.0
Capacity_2,NaN,"{'type': 'L', 'coefficients': {'Open_2': -70.0...",0.0


In [21]:
analyze_mps_high_level(file_path)

{'objective_function': {'name': 'TotalCost', 'num_coefficients': 30},
 'constraints': {'total': 36,
  'types': {'equality': 0, 'less_than_equal': 30, 'greater_than_equal': 6}},
 'rhs_values': {'total': 36,
  'sample': [('Demand_0', 1.0),
   ('Demand_1', 1.0),
   ('Demand_2', 1.0),
   ('Demand_3', 1.0),
   ('Demand_4', 1.0)]}}

In [26]:
import gzip
import pickle
from pathlib import Path
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric
from utils import load_gzip, load_json


class BipartiteNodeData(torch_geometric.data.Data):
    """
    This class encode a node bipartite graph observation as returned by the ecole.observation.NodeBipartite
    observation function in a format understood by the pytorch geometric data handlers.
    """

    def __init__(
        self,
        constraint_features,
        edge_indices,
        edge_features,
        variable_features,
        candidates,
        nb_candidates,
        candidate_choice,
        candidate_scores,
    ):
        super().__init__()
        self.constraint_features = constraint_features
        self.edge_index = edge_indices
        self.edge_attr = edge_features
        self.variable_features = variable_features
        self.candidates = candidates
        self.nb_candidates = nb_candidates
        self.candidate_choices = candidate_choice
        self.candidate_scores = candidate_scores

    def __inc__(self, key, value, store, *args, **kwargs):
        """
        We overload the pytorch geometric method that tells how to increment indices when concatenating graphs
        for those entries (edge index, candidates) for which this is not obvious.
        """
        if key == "edge_index":
            return torch.tensor(
                [[self.constraint_features.size(0)], [self.variable_features.size(0)]]
            )
        elif key == "candidates":
            return self.variable_features.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)


class GraphDataset(torch_geometric.data.Dataset):
    """
    This class encodes a collection of graphs, as well as a method to load such graphs from the disk.
    It can be used in turn by the data loaders provided by pytorch geometric.
    """

    def __init__(self, sample_files, edge_nfeats=2):
        super().__init__(root=None, transform=None, pre_transform=None)
        self.sample_files = sample_files
        self.edge_nfeats = edge_nfeats

    def len(self):
        return len(self.sample_files)

    def get(self, index, nan_mask_val=0):
        """
        This method loads a node bipartite graph observation as saved on the disk during data collection.
        """
        sample = load_gzip(self.sample_files[index])
        sample_observation, sample_action, sample_action_set, sample_scores = sample
        
        constraint_features = sample_observation.row_features
        edge_indices = sample_observation.edge_features.indices.astype(np.int32)
        edge_features = np.expand_dims(sample_observation.edge_features.values, axis=-1)
        if self.edge_nfeats == 2:
            edge_features_norm = edge_features / np.linalg.norm(edge_features) 
            edge_features = np.concatenate((edge_features, edge_features_norm), axis=-1)
        variable_features = sample_observation.variable_features

        constraint_features = np.nan_to_num(constraint_features, nan=nan_mask_val)
        edge_features = np.nan_to_num(edge_features, nan=nan_mask_val)
        variable_features = np.nan_to_num(variable_features, nan=nan_mask_val)

        # We note on which variables we were allowed to branch, the scores as well as the choice
        # taken by strong branching (relative to the candidates)
        candidates = np.array(sample_action_set, dtype=np.int32)
        candidate_scores = np.array([sample_scores[j] for j in candidates])
        candidate_choice = np.where(candidates == sample_action)[0][0]

        graph = BipartiteNodeData(
            torch.FloatTensor(constraint_features),
            torch.LongTensor(edge_indices),
            torch.FloatTensor(edge_features),
            torch.FloatTensor(variable_features),
            torch.LongTensor(candidates),
            len(candidates),
            torch.LongTensor([candidate_choice]),
            torch.FloatTensor(candidate_scores)
        )

        # We must tell pytorch geometric how many nodes there are, for indexing purposes
        graph.num_nodes = constraint_features.shape[0] + variable_features.shape[0]

        return graph

ImportError: cannot import name 'load_gzip' from 'utils' (c:\Users\timpi\Documents\Ugent\Masterproef\multi_agent_supply_chain_optimization\utils.py)

In [28]:
import torch
from torch_geometric.data import Data

def load_pickle(path):
    with open(path, 'rb') as file:
        obj = pickle.load(file)
    return obj

obj = load_pickle("data_0.pkl")

print(obj)

data = torch.load()  # returns a Data (MyData) object
print(data.x_rows.shape, data.edge_index_rowcols.shape, data.label)


ModuleNotFoundError: No module named 'gap_data'

In [34]:
# load_data.py
import sys
import types
import torch
from torch_geometric.data import Data

# 1) Create a fake module named gap_data with the bare minimum class
gap_data = types.ModuleType("gap_data")
class MyData(Data):
    """Stub for the unpickled object."""
    pass
gap_data.MyData = MyData
# If there’s also MyDataWithLabels, stub it too:
gap_data.MyDataWithLabels = type("MyDataWithLabels", (Data,), {})

# 2) Inject that module into sys.modules so torch.load can find it
sys.modules["gap_data"] = gap_data

def load_pickle(path):
    with open(path, 'rb') as file:
        obj = pickle.load(file)
    return obj

obj = load_pickle("data_0.pkl")

print(obj)


print(obj.x_rows.shape)              # torch.Size([70, 29])
print(obj.x_cols.shape)              # torch.Size([824, 17])
print(obj.edge_index_rowcols.shape)  # torch.Size([2, 1648])
print(obj.edge_vals_rowcols.shape)   # torch.Size([1648, 2])

print(hasattr(obj, "label"), getattr(obj, "label", None))


MyData(x_rows=[70, 29], x_cols=[824, 17], edge_index_rowcols=[2, 1648], edge_vals_rowcols=[1648, 2])
torch.Size([70, 29])
torch.Size([824, 17])
torch.Size([2, 1648])
torch.Size([1648, 2])
False None


In [37]:
# load_data.py
import sys
import types
import pickle
import torch
import numpy as np
from torch_geometric.data import Data

# 1) Stub the pickled module & classes
gap_data = types.ModuleType("gap_data")
class MyData(Data): pass
gap_data.MyData = MyData
gap_data.MyDataWithLabels = type("MyDataWithLabels", (Data,), {})
sys.modules["gap_data"] = gap_data

# 2) Unpickle with pickle.load
with open("data_0.pkl", "rb") as f:
    data = pickle.load(f)

# 3) Convert tensors to numpy
x_rows   = data.x_rows.numpy()               # [#rows, row_feat_dim]
x_cols   = data.x_cols.numpy()               # [#cols, col_feat_dim]
edge_idx = data.edge_index_rowcols.numpy()   # [2, E]
edge_val = data.edge_vals_rowcols.numpy()    # [E, 2]  (take [:,0] for raw coeffs)

num_rows, row_dim = x_rows.shape
num_cols, col_dim = x_cols.shape

# 4) Build dense adjacency matrix using the raw coefficient (col 0 of edge_val)
adj = np.zeros((num_rows, num_cols), dtype=float)
for r, c, v in zip(edge_idx[0], edge_idx[1], edge_val[:,0]):
    adj[r, c] = v

# 5) Print out adjacency snippet
print(f"Adjacency matrix: {num_rows}×{num_cols}")
print("Top‑left 5×10 block of adj (raw coefficients):")
print(adj[:5, :10])

# 6) Flatten feature matrices into one long vector
vec_rows = x_rows.reshape(-1)    # length = num_rows * row_dim
vec_cols = x_cols.reshape(-1)    # length = num_cols * col_dim
full_vec = np.concatenate([vec_rows, vec_cols])
print(f"\nFeature vector length: {full_vec.size}")
print("First 20 elements of feature vector:")
print(full_vec[:20])


Adjacency matrix: 70×824
Top‑left 5×10 block of adj (raw coefficients):
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Feature vector length: 16038
First 20 elements of feature vector:
[0.         0.         1.         0.         0.         0.
 1.         0.         0.         0.         0.14142136 0.
 0.         0.         0.06067961 0.         0.         0.06067961
 1.         0.        ]
